# Codon models

## Substitution models for protein-coding sequences

Les mutations non-synonymes etant plus deletaires que les mutations synonymes, ont tendance à etre non selectionnées.

In [1]:
data <- readDiscreteCharacterData("/home/tommaso/M2/Semestre_3/Phylo/TP4/placZFX20.nex")
data_codon = data.translateCharacters("Codon")

   Missing Variable:	Variable os does not exist

   Successfully read one character matrix from file '/home/tommaso/M2/Semestre_3/Phylo/TP4/placZFX20.nex'


In [2]:
print(data)


DNA character matrix with 20 taxa and 204 characters
Origination:                   placZFX20.nex
Number of taxa:                20
Number of included taxa:       20
Number of characters:          204
Number of included characters: 204
Datatype:                      DNA




In [3]:
print(data_codon)


CODON character matrix with 20 taxa and 68 characters
Origination:                   
Number of taxa:                20
Number of included taxa:       20
Number of characters:          68
Number of included characters: 68
Datatype:                      CODON




In [4]:
# Model parameters
omega = 0.2
kappa ~ dnExponential(lambda = 0.1)

In [5]:
Q := fnCodonHKY(omega=omega, kappa=kappa, baseFrequencies=nucstat)

   Missing Variable:	Variable nucstat does not exist


In [9]:
n_species <- data.ntaxa()
n_branches <- 2 * n_species - 3
taxa <- data.taxa()

out_group = clade("Sorex")

# tree topology
topology ~ dnUniformTopology(taxa, outgroup = out_group) 

for (i in 1:n_branches) {
 bl[i] ~ dnExponential(10.0)
}
psi := treeAssembly(topology, bl)

# total tree length (just for monitoring)
TL := sum(bl)


# nucstat
nucstat ~ dnDirichlet([1.0, 1.0, 1.0, 1.0])

# kappa
kappa ~ dnexponential(lambda = 0.1)

# omega
omega ~ dnexponential(lambda = 5)

Q := fnCodonHKY(omega=omega, kappa=kappa, baseFrequencies=nucstat)
seq ~ dnPhyloCTMC( tree=psi, Q=Q, type="Codon" )
seq.clamp( data_codon )


my_model = model(psi)


# moves
moves = VectorMoves()
moves.append(mvNNI(topology, weight=3.0))
moves.append(mvSPR(topology, weight=3.0))
moves.append(mvScale(kappa, weight=1.0))
moves.append(mvDirichletSimplex(nucstat, weight=1.0, alpha=10))
moves.append(mvScale(omega), weight=1.0)
for (i in 1:n_branches) {
   moves.append(mvScale(bl[i], weight=1.0))
}

# monitors
monitors[1] = mnModel(filename="../TP4/analyses/placZFX20_codon.log", printgen=10, separator = TAB)
monitors[2] = mnFile(psi,filename="../TP4/analyses/placZFX20_codon.trees", printgen=10, separator = TAB)
monitors[3] = mnScreen(printgen=100, TL)

# Make the analysis object
analysis = mcmc(my_model, monitors, moves)
# Run the MCMC
analysis.run(5000)

# Post-analysis: constructing the maximum a posteriori tree
treetrace = readTreeTrace("../TP4/analyses/placZFX20_codon.trees", treetype="non-clock", burnin=0.1)
map_tree = mapTree(treetrace, "../TP4/analyses/placZFX20_codon_map.tree")

   Error:	No function named 'dnexponential'
   Error:	No function named 'dnexponential'
   Missing Variable:	Variable kappa does not exist
   Missing Variable:	Variable Q does not exist
   Missing Variable:	Variable seq does not exist
   Missing Variable:	Variable kappa does not exist
   Error:	Argument or label mismatch for function call.
   Provided call:
   mvScale (Probability<constant> )
   
   Correct usage is:
   mvScale (Real<stochastic> x,
            RealPos<any> lambda,
            Bool<any> tune,
            RealPos<any> weight,
            Probability<any> tuneTarget)
   
   Error:	Cannot find node with name 'nucstat' in the model but received a move working on it.
   Missing Variable:	Variable analysis does not exist
   Error:	Could not find filename: placZFX20_codon.trees
   Could not read file named "placZFX20_codon.trees" in directory named "/home/tommaso/revbayes_kernel/../TP4/analyses" because neither the directory nor the
   file exist
   Missing Variable:	Variable 